# Projeto de Machine Learning para prever quem sobreviveria no Titanic

---



In [ ]:
import pandas as pd

In [ ]:
# Importando a base
df = pd.read_csv('/content/drive/MyDrive/Data Science/Titanic/train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [ ]:
df.shape

(891, 12)

In [ ]:
# Importando base teste
X_teste = pd.read_csv('/content/drive/MyDrive/Data Science/Titanic/test.csv')

# Base utilizada no final do código para gerar o arquivo final com a predição e os passageiros
X_teste2 = X_teste.loc[:, ['PassengerId', 'Name']]

X_teste

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


## **Nesta parte do código, vou tratar os valores nulos (caso haja necessidade)**

In [ ]:
# Somando todos os campos onde são nulos para a validação
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [ ]:
X_teste.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [ ]:
# Verificando dois casos onde o embarque esta vazio.
df.loc[pd.isnull(df['Embarked'])] 

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
61,62,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,NaN
829,830,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,NaN


Obs.: Apesar de ambas passageiras terem sobrevivido, decidi retirar estes dois dados por não ter certeza se realmente embarcaram no navio. Desta forma, posso realizar o modelo de machine learning apenas para as pessoas que com certeza entraram no navio

In [ ]:
# Excluindo ambos registros pelo index
df2 = df.drop(labels=[61, 829], axis=0)

# Conferindo se deu certo
df2.loc[pd.isnull(df['Embarked'])] 

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked


Por ter muitos dados em branco da cabine, vou desconsiderar esta informação. Agora para os passageiros onde possuem idade vazia vou assumir a média de idade dos passageiros para os mesmos. Desta forma, poderemos utilizar este dado no modelo.


In [ ]:
df2.loc[pd.isnull(df['Age'])] 

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
17,18,1,2,"Williams, Mr. Charles Eugene",male,NaN,0,0,244373,13.0000,NaN,S
19,20,1,3,"Masselmani, Mrs. Fatima",female,NaN,0,0,2649,7.2250,NaN,C
26,27,0,3,"Emir, Mr. Farred Chehab",male,NaN,0,0,2631,7.2250,NaN,C
28,29,1,3,"O'Dwyer, Miss. Ellen ""Nellie""",female,NaN,0,0,330959,7.8792,NaN,Q
...,...,...,...,...,...,...,...,...,...,...,...,...
859,860,0,3,"Razi, Mr. Raihed",male,NaN,0,0,2629,7.2292,NaN,C
863,864,0,3,"Sage, Miss. Dorothy Edith ""Dolly""",female,NaN,8,2,CA. 2343,69.5500,NaN,S
868,869,0,3,"van Melkebeke, Mr. Philemon",male,NaN,0,0,345777,9.5000,NaN,S
878,879,0,3,"Laleff, Mr. Kristo",male,NaN,0,0,349217,7.8958,NaN,S


In [ ]:
# Preenchendo os espaços de idade em branco pela média 
df2['Age'].fillna(df2['Age'].mean(), inplace=True)

# Preenchendo valores nulos no DataFrame teste
X_teste['Age'].fillna(X_teste['Age'].mean(), inplace=True)
X_teste['Fare'].fillna(X_teste['Fare'].mean(), inplace=True) # Preenchi o único registro que estava vazio com a média da tarifa

# Checando dois registros de exemplo se esta correto
df2.loc[(df2['PassengerId'] == 6) | (df2['PassengerId'] == 18)]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
5,6,0,3,"Moran, Mr. James",male,29.642093,0,0,330877,8.4583,NaN,Q
17,18,1,2,"Williams, Mr. Charles Eugene",male,29.642093,0,0,244373,13.0000,NaN,S


In [ ]:
# Somando todos os campos onde são nulos para a validação
df2.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
dtype: int64

Nesta etapa, vou separar os valores em dois eixos X e Y. Onde serão armazenadas
- X - teste : Recurso para a testagem do modelo/ Features
- X - treino: Recurso para treinamento do modelo/ Features
- Y - treino : Resultado final do treinamento do modelo/ Result

In [ ]:
# Separando as colunas que não serão necessárias no modelo e o resultado final que será separado abaixo no eixo Y
X_train = df2.drop(['PassengerId','Survived', 'Name', 'Ticket', 'Cabin'], axis=1).values

X_train, X_train.shape

(array([[3, 'male', 22.0, ..., 0, 7.25, 'S'],
        [1, 'female', 38.0, ..., 0, 71.2833, 'C'],
        [3, 'female', 26.0, ..., 0, 7.925, 'S'],
        ...,
        [3, 'female', 29.64209269662921, ..., 2, 23.45, 'S'],
        [1, 'male', 26.0, ..., 0, 30.0, 'C'],
        [3, 'male', 32.0, ..., 0, 7.75, 'Q']], dtype=object), (889, 7))

In [ ]:
# Separando os mesmos valores **BASE TESTE**
X_teste = X_teste.drop(['PassengerId','Name', 'Ticket','Cabin'], axis=1).values

X_teste, X_teste.shape

(array([[3, 'male', 34.5, ..., 0, 7.8292, 'Q'],
        [3, 'female', 47.0, ..., 0, 7.0, 'S'],
        [2, 'male', 62.0, ..., 0, 9.6875, 'Q'],
        ...,
        [3, 'male', 38.5, ..., 0, 7.25, 'S'],
        [3, 'male', 30.272590361445783, ..., 0, 8.05, 'S'],
        [3, 'male', 30.272590361445783, ..., 1, 22.3583, 'C']],
       dtype=object), (418, 7))

In [ ]:
# Selecionando apenas a coluna de resultado
Y_train = df2.loc[:, 'Survived'].values

Y_train

array([0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1,
       1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1,
       0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,

**Label Encoder** - nesta etapa vou reclassificar os campos que possuem strings para registros numéricos. Com isso, o modelo de machine de learning poderá considerar os campos.  



In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
# Vendo quais são os índices/valores que iremos utilizar o LabelEncoder
X_train[0]

array([3, 'male', 22.0, 1, 0, 7.25, 'S'], dtype=object)

In [ ]:
# Os campos que mudaremos são aqueles que são strings no caso: Sex/1 e Embarked/6
label_encoder_treino = LabelEncoder()

In [ ]:
# Assumindo os valores numéricos no lugar da string
X_train[:, 1] = label_encoder_treino.fit_transform(X_train[:, 1])
X_train[:, 6] = label_encoder_treino.fit_transform(X_train[:, 6])

In [ ]:
# Checagem se esta de acordo
X_train[0]

array([3, 1, 22.0, 1, 0, 7.25, 2], dtype=object)

In [ ]:
# Os campos que mudaremos são aqueles que são strings no caso: Sex/1 e Embarked/6 - **BASE TESTE**
label_encoder_teste = LabelEncoder()

In [ ]:
# Assumindo os valores numéricos no lugar da string - **BASE TESTE**
X_teste[:, 1] = label_encoder_teste.fit_transform(X_teste[:, 1])
X_teste[:, 6] = label_encoder_teste.fit_transform(X_teste[:, 6])

In [ ]:
# Validando as alterações **BASE TESTE**
X_teste

array([[3, 1, 34.5, ..., 0, 7.8292, 1],
       [3, 0, 47.0, ..., 0, 7.0, 2],
       [2, 1, 62.0, ..., 0, 9.6875, 1],
       ...,
       [3, 1, 38.5, ..., 0, 7.25, 2],
       [3, 1, 30.272590361445783, ..., 0, 8.05, 2],
       [3, 1, 30.272590361445783, ..., 1, 22.3583, 0]], dtype=object)

# **Implementando o modelo**

Para este projeto, decidi utilizar Logistic Regression pois é um modelo onde se encaixa perfeitamente neste caso de classificação e é um modelo onde tenho mais conhecimento de como funciona.

In [ ]:
from sklearn.linear_model import LogisticRegression

# Treinando o modelo
logisticRegr = LogisticRegression()
logisticRegr.fit(X_train, Y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression()

In [ ]:
# Prevendo o resultado
predictions = logisticRegr.predict(X_teste)

output = pd.DataFrame({'PassengerId': X_teste2.PassengerId,'Survived': predictions})

In [ ]:
# Checando a base final do modelo
output

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [ ]:
# Salvando a base final para realizar o upload no Kaggle
output.to_csv('/content/drive/MyDrive/Data Science/Titanic/titanic1.csv', index=False)

Após submeter o modelo do Kagle, tive o retorno de **Score = 0.76794**